In [1]:
import pandas as pd
from sqlalchemy import create_engine
import getpass

pd.options.display.max_rows = 4

### Connect to databases.

MySQL is the old database hosted on Dreamhost.

Postgres is the new database on localhost.

In [2]:
password = getpass.getpass('Database password: ')

Database password: ········


In [7]:
engine_postgres = create_engine('postgres://aubreymoore:{}@localhost/import_test'.format(password))
engine_mysql = create_engine('mysql+pymysql://aubreymoore:{}@mysql.guaminsects.net/pestlist'.format(password))

print 'postgres'
print engine_postgres.table_names()
print
print 'mysql'
print engine_mysql.table_names()
#pd.read_sql_table('taxon', engine)

postgres
[u'associate2', u'auth_cas', u'auth_event', u'auth_group', u'auth_membership', u'auth_permission', u'auth_user', u'bugwood', u'extracted_names', u'factsheet', u'location_recode', u'pacific_pests', u'resolved_names', u'syn', u'synonym2', u'taxon2', u'vernacular', u'test', u'person', u'old_new', u'taxon_temp', u'associate_temp', u'image', u'tax', u'name_biblio', u'biblio', u'biblio_type', u'taxonx', u'associate', u'xgeo', u'geo', u'name_type', u'name', u'geo_biblio', u'name1', u'distribution', u'tax1', u'taxon1', u'taxon']

mysql
[u'associate2', u'auth_cas', u'auth_event', u'auth_group', u'auth_membership', u'auth_permission', u'auth_user', u'bugwood', u'extracted_names', u'factsheet', u'geo', u'location_recode', u'pacific_pests', u'resolved_names', u'syn', u'synonym2', u'taxon2', u'vernacular']


### Create a csv import file for the **associate** table.

In [4]:
df_associate2 = pd.read_sql_table('associate2', engine_mysql)
df_associate2

,id,t1,t2,type,note
0,1,7,326,plant host|herbivore,None
1,2,7,342,plant host|herbivore,None
...,...,...,...,...,...
929,930,319,1037,plant host|herbivore,None
930,931,319,752,plant host|herbivore,None


In [5]:
df_taxon2 = pd.read_sql_table('taxon2', engine_mysql)
df_taxon2

,id,tid,parent_tid,name,trank,lineage
0,1,6,None,Plantae,kingdom,Plantae
1,2,7707728,6,Tracheophyta,phylum,Plantae|Tracheophyta
...,...,...,...,...,...,...
804,6107,3177662,5014,Eugenia,genus,Plantae|Tracheophyta|Magnoliopsida|Myrtales|My...
805,6108,7907394,3177662,Eugenia malaccensis,species,Plantae|Tracheophyta|Magnoliopsida|Myrtales|My...


In [6]:
# Get names for t1 and t2 in associate2 table
df = df_associate2.merge(right=df_taxon2, left_on='t1', right_on='id')
df = df.merge(right=df_taxon2, left_on='t2', right_on='id')
df = df[['name_x', 'name_y']]

# Replace names with ids from taxon table
df_taxon = pd.read_sql_table('taxon', engine_postgres)
df = df.merge(df_taxon, left_on='name_x', right_on='name')
df = df.merge(df_taxon, left_on='name_y', right_on='name')
df = df[['id_x', 'id_y']]
df.columns = ['associate.taxon1', 'associate.taxon2']

# Write CSV file and display data frame
df.to_csv('associate.csv', index=False)
df

,associate.taxon1,associate.taxon2
0,2315,1932
1,2429,1932
...,...,...
929,2412,1858
930,2412,2191


### Create a csv import file for the **image** table.